<a href="https://colab.research.google.com/github/joudelshawa/mental-health-ai-app/blob/feature%2Fdata_preprocessing/tweepy_real_time_twitter_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tweepy

In [ ]:
import tweepy
from pymongo import MongoClient

In [ ]:
# import modules
import pandas as pd
import tweepy
  
  
# function to display data of each tweet
def printtweetdata(n, ith_tweet):
    print()
    print(f"Tweet {n}:")
    print(f"Username:{ith_tweet[0]}")
    print(f"Description:{ith_tweet[1]}")
    print(f"Location:{ith_tweet[2]}")
    print(f"Following Count:{ith_tweet[3]}")
    print(f"Follower Count:{ith_tweet[4]}")
    print(f"Total Tweets:{ith_tweet[5]}")
    print(f"Retweet Count:{ith_tweet[6]}")
    print(f"Tweet Text:{ith_tweet[7]}")
    print(f"Hashtags Used:{ith_tweet[8]}")
  
  
# function to perform data extraction
def scrape(words, date_since, numtweet):
      
    # Creating DataFrame using pandas
    db = pd.DataFrame(columns=['username', 'description', 'location', 'following',
                               'followers', 'totaltweets', 'retweetcount', 'text', 'hashtags'])
      
    # We are using .Cursor() to search through twitter for the required tweets.
    # The number of tweets can be restricted using .items(number of tweets)
    tweets = tweepy.Cursor(api.search, q=words, lang="en",
                           since=date_since, tweet_mode='extended').items(numtweet)
     
    # .Cursor() returns an iterable object. Each item in 
    # the iterator has various attributes that you can access to 
    # get information about each tweet
    list_tweets = [tweet for tweet in tweets]
      
    # Counter to maintain Tweet Count
    i = 1  
      
    # we will iterate over each tweet in the list for extracting information about each tweet
    for tweet in list_tweets:
        username = tweet.user.screen_name
        description = tweet.user.description
        location = tweet.user.location
        following = tweet.user.friends_count
        followers = tweet.user.followers_count
        totaltweets = tweet.user.statuses_count
        retweetcount = tweet.retweet_count
        hashtags = tweet.entities['hashtags']
          
        # Retweets can be distinguished by a retweeted_status attribute,
        # in case it is an invalid reference, except block will be executed
        try:
            text = tweet.retweeted_status.full_text
        except AttributeError:
            text = tweet.full_text
        hashtext = list()
        for j in range(0, len(hashtags)):
            hashtext.append(hashtags[j]['text'])
          
        # Here we are appending all the extracted information in the DataFrame
        ith_tweet = [username, description, location, following,
                     followers, totaltweets, retweetcount, text, hashtext]
        db.loc[len(db)] = ith_tweet
          
        # Function call to print tweet data on screen
        printtweetdata(i, ith_tweet)
        i = i+1
    filename = 'scraped_tweets.csv'
      
    # we will save our database as a CSV file.
    db.to_csv(filename)
  
  
if __name__ == '__main__':
      
    # Enter your own credentials obtained 
    # from your developer account
    consumer_key = "mdmzMBrRM2nNnTdEoN7jZuxmY"
    consumer_secret = "3D6YYKcs6kheu1VjqZzkHOu9Fu5RBXKYOG947t0PGykr39btce"
    access_key = "525051045-gOq1C7h2YUbz1gFhRuehci0atHZUPwFC5nwlv0VP"
    access_secret = "eXvSLyGa9OrzS6R1EiZ86nxvHGpP0RsxelqyVWkxZqmYV"
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
      
    # Enter Hashtag and initial date
    print("Enter Twitter HashTag to search for")
    words = input()
    print("Enter Date since The Tweets are required in yyyy-mm--dd")
    date_since = input()
      
    # number of tweets you want to extract in one run
    numtweet = 100  
    scrape(words, date_since, numtweet)
    print('Scraping has completed!')

In [ ]:
consumer_key = "mdmzMBrRM2nNnTdEoN7jZuxmY"
consumer_secret = "3D6YYKcs6kheu1VjqZzkHOu9Fu5RBXKYOG947t0PGykr39btce"
access_token = "525051045-gOq1C7h2YUbz1gFhRuehci0atHZUPwFC5nwlv0VP"
access_token_secret = "eXvSLyGa9OrzS6R1EiZ86nxvHGpP0RsxelqyVWkxZqmYV"

In [ ]:
def get_all_tweets(screen_name):
    client = MongoClient()
    db = client.db_twitter
    tweets = db.Tweets

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
# initialize a list to hold all the tweepy Tweets
alltweets = []

# make initial request for most recent tweets (200 is the maximum allowed count)
new_tweets = api.user_timeline(screen_name=get_all_tweets, count=200)

# save most recent tweets
alltweets.extend(new_tweets)

# save the id of the oldest tweet less one
oldest = alltweets[-1].id - 1

# keep grabbing tweets until there are no tweets left to grab
while len(new_tweets) > 0:
    print
    "getting tweets before %s" % (oldest)

    # all subsiquent requests use the max_id param to prevent duplicates
    new_tweets = api.user_timeline(screen_name=get_all_tweets, count=200, max_id=oldest)

    # save most recent tweets
    alltweets.extend(new_tweets)

    # update the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

   # print("...%s tweets downloaded so far" % (len(alltweets)))

# transform the tweepy tweets into a 2D array that will populate the csv
outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets]

In [ ]:
import os
import tweepy as tw
import pandas as pd
import tweepy  
import csv
import datetime
consumer_key = "mdmzMBrRM2nNnTdEoN7jZuxmY"
consumer_secret = "3D6YYKcs6kheu1VjqZzkHOu9Fu5RBXKYOG947t0PGykr39btce"
access_key = "525051045-gOq1C7h2YUbz1gFhRuehci0atHZUPwFC5nwlv0VP"
access_secret = "eXvSLyGa9OrzS6R1EiZ86nxvHGpP0RsxelqyVWkxZqmYV"
api = tweepy.API(auth)
#Add time your file was created to discriminate date/time after running the crontab file
#filename = 'twitter_data_analysis'+(datetime.datetime.now().strftime("%Y-%m-%d-%H"))+'.csv'
#with open (filename, 'a+', newline='') as csvFile:
 #  csvWriter = csv.writer(csvFile)

In [ ]:
#auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_key, access_secret)
#api = tweepy.API(auth)
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
# Define the search term and the date_since date as variables
search_words = "#wildfires"
date_since = "2018-11-16"

In [ ]:
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(5)
tweets

In [ ]:
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(5)

# Iterate and print tweets
for tweet in tweets:
    print(tweet.text)

In [ ]:
# Collect tweets
tweets = tw.Cursor(api.search,
                       q=search_words,
                       lang="en",
                       since=date_since).items(5)

# Collect a list of tweets
[tweet.text for tweet in tweets]